<small>Copyright 2025 Amazon.com, Inc. or its affiliates. All Rights Reserved.<br>
This is AWS Content subject to the terms of the Customer Agreement</small>

# Module 3.2: Video Quality Evaluation Using LLM as Judge

This notebook demonstrates the video quality evaluation pipeline using Amazon Nova Premier to assess video quality across multiple technical and aesthetic dimensions. We'll explore how to evaluate temporal consistency, aesthetic quality, technical quality, and motion effects.

## A. Overview

The video quality evaluation pipeline consists of:
1. **Quality Metric Evaluation** - Assess videos across 4 key quality dimensions
2. **LLM as Judge** - Use multimodal AI to score quality on 1-5 scale
3. **Structured Scoring** - Extract scores and justifications for each metric
4. **Complete Pipeline** - End-to-end quality assessment with S3 storage

## B. Install Dependencies

First, let's install the required packages for this notebook.

In [ ]:
!pip install -q matplotlib opencv-python Pillow tqdm

## C. Setup and Imports

In [ ]:
import boto3
import json
import time
from botocore.exceptions import ClientError

from utils.quality_assessment import evaluate_video_quality_metric, video_quality_evaluation_pipeline, TECHNICAL_QUALITY_PROMPT, AESTHETIC_QUALITY_PROMPT, TEMPORAL_CONSISTENCY_PROMPT, MOTION_EFFECTS_PROMPT
from utils.config import get_s3_bucket, discover_video_files

## C. Configuration

Configure your AWS session and evaluation parameters. Make sure your video files exist in S3.

- S3 bucket will be automatically detected from CloudFormation or fallback to config.json
- Video generated from previous steps will be automatically detected and the first video will be used for the evaluation

In [ ]:
# AWS Configuration
session = boto3.Session()

# Get S3 bucket name
S3_BUCKET = get_s3_bucket(session)

# Load configuration for video prefix
with open('config.json', 'r') as f:
    config = json.load(f)

# Video quality evaluation parameters - update with your video name

available_videos = discover_video_files(
    session, 
    S3_BUCKET, 
    config['video_prefix']
)
VIDEO_NAME = available_videos[0]
S3_VIDEO_URI = f"s3://{S3_BUCKET}/{config['video_prefix']}{VIDEO_NAME}"
MODEL_ID = "us.amazon.nova-premier-v1:0"

# Quality metrics to evaluate
QUALITY_METRICS = [
    "temporal_consistency",
    "aesthetic_quality", 
    "technical_quality",
    "motion_effects"
]

## D. Step 1 - Single Quality Metric Evaluation

The first step evaluates a video against a specific quality metric using LLM as judge. The `evaluate_video_quality_metric` function:

**Key Operations:**
1. **Multimodal analysis** - Processes video content with quality-specific prompts
2. **Structured scoring** - Returns 1-5 scale scores with justifications
3. **Quality assessment** - Evaluates technical and aesthetic aspects
4. **Error handling** - Gracefully handles API failures and parsing errors

**Quality Metrics:**
- **Temporal Consistency**: Frame-to-frame coherence and smoothness
- **Aesthetic Quality**: Visual composition, color harmony, artistic merit
- **Technical Quality**: Image sharpness, clarity, absence of artifacts
- **Motion Effects**: Natural movement patterns and camera work

- <font color='red'>Please feel free to use `print(TEMPORAL_CONSISTENCY_PROMPT)` to check the evaluation prompt used in this pipeline</font>

In [ ]:
# Test temporal consistency evaluation
print("🎬 Evaluating temporal consistency...")

score, justification = evaluate_video_quality_metric(
    boto3_session=session,
    s3_video_uri=S3_VIDEO_URI,
    metric_prompt=TEMPORAL_CONSISTENCY_PROMPT,
    model_id=MODEL_ID
)

print(f"\n📊 Temporal Consistency Score: {score}/5")
print(f"💭 Justification: {justification}")

## E. Step 2 - Aesthetic Quality Assessment

Now we evaluate the aesthetic quality of the video, focusing on visual composition, color harmony, and artistic elements.

- <font color='red'>Please feel free to use `print(AESTHETIC_QUALITY_PROMPT)` to check the evaluation prompt used in this pipeline</font>

In [ ]:
# Test aesthetic quality evaluation
print("🎨 Evaluating aesthetic quality...")

score, justification = evaluate_video_quality_metric(
    boto3_session=session,
    s3_video_uri=S3_VIDEO_URI,
    metric_prompt=AESTHETIC_QUALITY_PROMPT,
    model_id=MODEL_ID
)

print(f"\n🎭 Aesthetic Quality Score: {score}/5")
print(f"💭 Justification: {justification}")

## F. Step 3 - Technical Quality Assessment

Evaluate the technical aspects of the video including sharpness, clarity, and absence of visual artifacts.

- <font color='red'>Please feel free to use `print(TECHNICAL_QUALITY_PROMPT)` to check the evaluation prompt used in this pipeline</font>

In [ ]:
# Test technical quality evaluation
print("🔧 Evaluating technical quality...")

score, justification = evaluate_video_quality_metric(
    boto3_session=session,
    s3_video_uri=S3_VIDEO_URI,
    metric_prompt=TECHNICAL_QUALITY_PROMPT,
    model_id=MODEL_ID
)

print(f"\n⚙️ Technical Quality Score: {score}/5")
print(f"💭 Justification: {justification}")

## G. Step 4 - Motion Effects Assessment

Evaluate the quality of movement and dynamics in the video, including natural motion patterns and smooth camera work.

- <font color='red'>Please feel free to use `print(MOTION_EFFECTS_PROMPT)` to check the evaluation prompt used in this pipeline</font>

In [ ]:
# Test motion effects evaluation
print("🏃 Evaluating motion effects...")

score, justification = evaluate_video_quality_metric(
    boto3_session=session,
    s3_video_uri=S3_VIDEO_URI,
    metric_prompt=MOTION_EFFECTS_PROMPT,
    model_id=MODEL_ID
)

print(f"\n🎯 Motion Effects Score: {score}/5")
print(f"💭 Justification: {justification}")

## H. Complete Quality Evaluation Pipeline

The `video_quality_evaluation_pipeline` function demonstrates the complete evaluation workflow across all quality metrics. This wrapper function:

**Key Operations:**
1. **Multi-metric evaluation** - Processes all 4 quality dimensions automatically
2. **S3 integration** - Saves quality scores and justifications to S3
3. **Structured results** - Returns comprehensive quality assessment
4. **Error handling** - Gracefully handles API failures

**Output Structure:**
```
s3://<bucket>/
└── generated_videos/
    ├── <video_id>.mp4
    └── <video_id>_quality.json      # Quality scores and justifications
```

In [ ]:
print("🔧 Using production pipeline from utils/vid_eval2.py...")

# Run the production pipeline
production_results = video_quality_evaluation_pipeline(
    s3_video_uri=S3_VIDEO_URI,
    boto3_session=session,
    model_id=MODEL_ID,
    temporal_consistency_flag=True if "temporal_consistency" in QUALITY_METRICS else False,
    aesthetic_quality_flag=True if "aesthetic_quality" in QUALITY_METRICS else False,
    technical_quality_flag=True if "technical_quality" in QUALITY_METRICS else False,
    motion_effects_flag=True if "motion_effects" in QUALITY_METRICS else False
)

print(f"\n✅ Production pipeline completed!")
print(f"📊 Results: {json.dumps(production_results, indent=2)}")

## Conclusion

You've successfully implemented a comprehensive video quality evaluation system that:

1. **Evaluates multiple quality dimensions** using LLM as judge across temporal, aesthetic, technical, and motion aspects
2. **Provides structured scoring** with 1-5 scale ratings and detailed justifications
3. **Offers production-ready pipeline** for batch video quality assessment
